In [1]:
!pip install ollama

In [24]:
df = pd.read_csv("commit_message (1).csv")
df=df

In [28]:
df[:30]

,url,original_message,message,original_label,is_good,is_bad,binary_label
0,https://github.com/junit-team/junit4/commit/12...,Mark ThreadGroups created by FailOnTimeout as ...,Mark ThreadGroups created by FailOnTimeout as ...,Why and What,True,False,1.0
1,https://github.com/junit-team/junit4/commit/43...,Use Google's Maven Central mirror \n \n We ar...,Use Google's Maven Central mirror\nWe are usin...,Why and What,True,False,1.0
2,https://github.com/junit-team/junit4/commit/6c...,Remove reference to obsolete mailing list,Remove reference to obsolete mailing list,No Why,False,False,NaN
3,https://github.com/junit-team/junit4/commit/46...,Deprecate Assert#assertThat \n \n The method ...,Deprecate Assert#assertThat\nThe method 'asser...,Why and What,True,False,1.0
4,https://github.com/junit-team/junit4/commit/8a...,Revert 'ExternalResource: declare after() to t...,Revert 'ExternalResource: declare after() to t...,Why and What,False,False,NaN
5,https://github.com/junit-team/junit4/commit/37...,spelling: unwieldy,spelling: unwieldy,Why and What,False,True,0.0
6,https://github.com/junit-team/junit4/commit/b9...,spelling: methods,spelling: methods,Why and What,False,True,0.0
7,https://github.com/junit-team/junit4/commit/a5...,spelling: builder,spelling: builder,Why and What,False,True,0.0
8,https://github.com/junit-team/junit4/commit/21...,spelling: assert,spelling: assert,Why and What,False,True,0.0
9,https://github.com/junit-team/junit4/commit/03...,style: remove space,style: remove space,Why and What,False,True,0.0


In [163]:
import pandas as pd
import csv
from ollama import chat
import time
from typing import List, Dict, Callable, Optional

# Types
PromptFunction = Callable[[str], str]
ModelResponse = str
ClassificationFunction = Callable[[str, str, Optional[dict]], ModelResponse]

def classify_commits(
    input_csv: str, 
    output_csv: str, 
    model: str = 'llama3.2:3b',
    prompt_generator: PromptFunction = None,
    classification_func: ClassificationFunction = None,
    model_options: dict = None,
    binary_classification: bool = False,
    is_fewshot=False
) -> None:
    """
    Classify commit messages using Ollama with customizable prompting techniques.
    
    Args:
        input_csv: Path to input CSV file containing commit data
        output_csv: Path to output CSV file where classifications will be stored
        model: Ollama model to use
        prompt_generator: Function that generates the prompt for classification
        classification_func: Function that handles the classification process
        model_options: Additional options to pass to the model (temperature, etc.)
        binary_classification: Boolean flag to determine if binary "isGood" classification should be used
    """
    # Set defaults if not provided
    if prompt_generator is None:
        prompt_generator = zero_shot_prompt
    if classification_func is None:
        classification_func = basic_classification
    if model_options is None:
        model_options = {"temperature": 0}
    
    # Read input CSV
    df = pd.read_csv(input_csv)
    df = df[:10]  # Limiting to 10 for testing
    
    # Create results list
    results = []
    
    # Track progress
    total_commits = len(df)
    print(f"Starting classification of {total_commits} commits...")
    
    for idx, row in df.iterrows():
        # Extract message
        message = row['message']
        
        # Generate prompt
        prompt = prompt_generator(message,is_fewshot)

        # Classify the message
        classification = classification_func(prompt, model, model_options)
      
        
        # Create result dictionary with all original columns plus the new classification
        result = row.to_dict()
        result['model_classification'] = classification
        
        # Add to results
        results.append(result)
        
        # Sleep briefly to avoid rate limiting
        time.sleep(0.5)
        
        # Print progress every 10 commits
        if (idx + 1) % 10 == 0 or idx == total_commits - 1:
            print(f"Processed {idx + 1}/{total_commits} commits ({(idx + 1) / total_commits * 100:.1f}%)")
    
    # Create output DataFrame and save to CSV
    output_df = pd.DataFrame(results)
    output_df.to_csv(output_csv, index=False)
    
    print(f"Classification complete. Results saved to {output_csv}")
    
       



def classify_binary(prompt: str, model: str, options: dict) -> str:
    """Classify if the commit is 'good' or 'not good'"""
    
    response = chat(
        model=model, 
        messages=[{"role": "user", "content": prompt}],
        options=options
    )
    
    # Extract the final classification from <pred> tags
    return extract_pred(response.message.content)


def extract_pred(response_text: str) -> str:
#     print(response_text)
    """Extract classification from the response wrapped in <pred> tags"""
    start_tag = "<pred>"
    end_tag = "</pred>"
    
    start = response_text.find(start_tag)
    end = response_text.find(end_tag)
    
    if start != -1 and end != -1:
        return response_text[start + len(start_tag):end].strip()
    return "classification error"





# ===== Prompt Generators =====

def zero_shot_prompt(commit_message: str,is_fewshot=False) -> str:
    
    """Generate a zero-shot prompt for classification"""
    
    if is_fewshot==True:
        examples=few_shot_prompt()
    else:
        examples="";
    
    return f"""
    Analyze the following commit message and classify it into exactly one of these categories:
    - "why and what": The commit explains both WHY the change was made and WHAT was changed
    - "neither why nor what": The commit explains neither WHY nor WHAT
    - "no why": The commit explains WHAT was changed but not WHY
    - "no what": The commit explains WHY a change was needed but not WHAT was changed
    
    {examples}
    
    Commit message: {commit_message}
    
    Classification (respond with only one of the four categories wrapped in <pred> tags):
    """

def zero_shot_prompt_binary(commit_message: str,is_fewshot=False) -> str:
    """Generate a zero-shot prompt for classification"""
    if is_fewshot==True:
        examples=few_shot_prompt_binary()
    else:
        examples="";
    return f"""
  Your task is to classify commit messages into one of two categories:
    -Good
    -Bad
    - The commit explains both WHY the change was made and WHAT was changed : good 
    - The commit explains neither WHY nor WHAT : bad
    - The commit explains WHAT was changed but not WHY : bad
    - The commit explains WHY a change was needed but not WHAT was changed : bad
    

    Commit message: {commit_message}
    
    Classification (respond with only one of the two categories wrapped in <pred> tags):
    """

def few_shot_prompt() -> str:
    """Generate a few-shot prompt with examples for classification"""
    return f"""
    
    
    Here are some examples:
    
    Example 1:
    Commit message: "Mark ThreadGroups created by FailOnTimeout as daemon groups Mark ThreadGroup created by FailOnTimeout as a daemon group. Previously, FailOnTimeout destroyed the ThreadGroup, which could cause race conditions if the ThreadGroup was referenced by other threads."
    Classification: <pred>why and what</pred>
    
    Example 2:
    Commit message: "Fix xyz"
    Classification: <pred>no what</pred>
    
    Example 3:
    Commit message: "Remove reference to mailing list"
    Classification: <pred>no why</pred>
    
    Example 4:
    Commit message: "Formatting"
    Classification: <pred>neither why nor what</pred>
    
    """

def few_shot_prompt_binary() -> str:
    """Generate a few-shot prompt with examples for classification"""
    return f"""
    
    Here are some examples:
    
    Example 1:
    Commit message: "Mark ThreadGroups created by FailOnTimeout as daemon groups Mark ThreadGroup created by FailOnTimeout as a daemon group. Previously, FailOnTimeout destroyed the ThreadGroup, which could cause race conditions if the ThreadGroup was referenced by other threads."
    Classification: <pred>good</pred>
    
    Example 2:
    Commit message: "Fix xyz"
    Classification: <pred>bad</pred>
    
    Example 3:
    Commit message: "Remove reference to mailing list"
    Classification: <pred>bad</pred>
    
    Example 4:
    Commit message: "Formatting"
    Classification: <pred>bad</pred>
    """


def few_shot_prompt_cot() -> str:
    """Generate a few-shot prompt with examples for classification"""
    return f"""
    
    
    Here are some examples:
    
    Example 1:
    Commit message: "Mark ThreadGroups created by FailOnTimeout as daemon groups Mark ThreadGroup created by FailOnTimeout as a daemon group. Previously, FailOnTimeout destroyed the ThreadGroup, which could cause race conditions if the ThreadGroup was referenced by other threads."
    Chain of thought: 
    Does this say why we are doing it? yes, FailOnTimeout destroyed the ThreadGroup, which could cause race conditions.
    Is it clear what we have to mark? yes, we have to mark ThreadGroups creaded by FailOnTimeOut
    Classification: <pred>why and what</pred>
    
    Example 2:
    Commit message: "Fix xyz"
    Chain of thought: 
    Does this say why we are doing it? yes, there is a reason implied. It is implied that xyz is broken because we have to fix it. Is it clear what we are supposed to fix? yes, we are supposed to fix xyz.
    Classification: <pred>no what</pred>
    
    Example 3:
    Commit message: "Remove reference to mailing list"
    Chain of thought:
    Does tihs say why we are removing the reference? No it is unclear. Does it say what we should do? Yes, it mentions that we have to remove the reference to mailing list.
    Classification: <pred>no why</pred>
    
    Example 4:
    Commit message: "Formatting"
    Chain of thought: 
    Does this message mention why we are formatting? No. Does this message mention what we are formatting? No. It does not mention anything about why or what. 
    Classification: <pred>neither why nor what</pred>
    
    """

def structured_prompt(commit_message: str, is_fewshot=False) -> str:
    """Generate a structured reasoning prompt for classification"""
    if is_fewshot==True:
        examples=few_shot_prompt()
    else:
        examples="";
    return f"""
    Your task is to classify the following commit message into one of four categories.
    
    
    Commit message: {commit_message}
    
    Classification categories:
    - "why and what": The commit explains both WHY the change was made and WHAT was changed
    - "neither why nor what": The commit explains neither WHY nor WHAT
    - "no why": The commit explains WHAT was changed but not WHY
    - "no what": The commit explains WHY a change was needed but not WHAT was changed
    
    {examples}
    
    Analysis:
    First, let's check if the commit explains WHAT was changed:
    [Your analysis of WHAT aspect]
    
    Second, let's check if the commit explains WHY the change was made:
    [Your analysis of WHY aspect]
    
    Based on this analysis, the classification is:
    """

def structured_prompt_binary(commit_message: str,is_fewshot=False) -> str:
    """Generate a structured reasoning prompt for classification"""
    if is_fewshot==True:
        examples=few_shot_prompt_binary()
    else:
        examples="";
    return f"""
    Your task is to classify the following commit message into one of two categories.
    
    
    Commit message: {commit_message}
    
    -good
    -bad
    
    - The commit explains both WHY the change was made and WHAT was changed : good 
    - The commit explains neither WHY nor WHAT : bad
    - The commit explains WHAT was changed but not WHY : bad
    - The commit explains WHY a change was needed but not WHAT was changed : bad
    
    {examples}
    
    Analysis:
    First, let's check if the commit explains WHAT was changed:
    [Your analysis of WHAT aspect]
    
    Second, let's check if the commit explains WHY the change was made:
    [Your analysis of WHY aspect]
    
    Based on this analysis, the classification is:
    """


def cot_prompt(commit_message: str, is_fewshot=False) -> str:
    """Generate a chain-of-thought prompt for classification"""
    
    if is_fewshot==True:
        examples=few_shot_prompt_cot()
    else:
        examples="";
    return f"""
    Analyze the following commit message and classify it into exactly one of these categories:
    - "why and what": The commit explains both WHY the change was made and WHAT was changed
    - "neither why nor what": The commit explains neither WHY nor WHAT
    - "no why": The commit explains WHAT was changed but not WHY
    - "no what": The commit explains WHY a change was needed but not WHAT was changed
    
    {examples}
    
    Commit message: {commit_message}
    
    Let's think through this step by step:
    1. Does this commit explain WHAT was changed?
    2. Does this commit explain WHY the change was made?
    3. Based on the answers to questions 1 and 2, which category should this be classified as?

    Final classification (respond with only one of the four categories):
    """

def cot_prompt_binary(commit_message: str, is_fewshot=False) -> str:
    """Generate a chain-of-thought prompt for classification"""
    
    if is_fewshot==True:
        examples=few_shot_prompt_binary()
    else:
        examples="";
    return f"""
     Your task is to classify the following commit message into one of two categories.

    
    -good
    -bad
    
    - The commit explains both WHY the change was made and WHAT was changed : good 
    - The commit explains neither WHY nor WHAT : bad
    - The commit explains WHAT was changed but not WHY : bad
    - The commit explains WHY a change was needed but not WHAT was changed : bad
    
    {examples}
    
    Commit message: {commit_message}
    
    Let's think through this step by step:
    1. Does this commit explain WHAT was changed?
    2. Does this commit explain WHY the change was made?
    3. Based on the answers to questions 1 and 2, which category should this be classified as?

    Final classification (respond with only one of the four categories):
    """

# ===== Classification Functions =====

def basic_classification(prompt: str, model: str, options: dict) -> str:
    """Basic classification that calls the model and normalizes the output"""
    try:
        response = chat(
            model=model, 
            messages=[{"role": "user", "content": prompt}],
            options=options
        )
        
        classification = extract_pred(response.message.content)
        return classification
    
    except Exception as e:
        print(f"Error classifying commit: {e}")
        return "classification error"


def run_experiments(
    input_csv: str,
    experiment_configs: List[Dict],
    base_output_name: str = "commits_classified"
) -> None:
    """
    Run multiple classification experiments with different configurations, 
    generating output file names with hyperparameter details.
    
    Args:
        input_csv: Path to input CSV
        experiment_configs: List of experiment configurations
        base_output_name: Base name for output files
    """
    for i, config in enumerate(experiment_configs):
        # Construct the output name using the hyperparameters
        model_name = config.get("model", 'llama3.2:3b').replace(':', '_')
        temperature = config.get("model_options", {}).get("temperature", 0)
        binary_classification = config.get("binary_classification", False)
        is_fewshot= config.get('classfication_func')
        is_fewshot= config.get('is_fewshot')
        prompt_type=config.get('prompt_type')
        classification_type = "binary" if binary_classification else "multi-class"
        shot= "few" if is_fewshot else "zero"
        
        # Generate the file name
        output_name = f"{base_output_name}_{model_name}_{prompt_type}.csv"
        
        # Run the classification for the current configuration
        classify_commits(
            input_csv=input_csv,
            output_csv=output_name,
            model=config.get("model", 'llama3.2:3b'),
            prompt_generator=config.get("prompt_generator"),
            classification_func=config.get("classification_func"),
            model_options=config.get("model_options", {}),
            binary_classification=config.get("binary_classification", False),
            is_fewshot=is_fewshot
        )



In [164]:
def main():
    """Example usage with different prompting techniques"""
    # Input file
    input_csv = "commit_message (1).csv"
    
    # Define experiment configurations
    experiment_configs = [
    {
        "model": "llama3.2:3b",  # Using the 3B version of the model
        "prompt_generator": zero_shot_prompt,  # Zero-shot classification
        "prompt_type":"_basic_zeroshot",
        "classification_func": basic_classification,  # Basic classification function
        "model_options": {"temperature": 0},  # Lower temperature for deterministic output
        "binary_classification": False, # 4-class classification (not binary)
        "is_fewshot":False
    },
        {
        "model": "llama3.2:3b",  # Using the 3B version of the model
        "prompt_generator": zero_shot_prompt,  # Zero-shot classification
        "prompt_type":"_basic_fewshot",
        "classification_func": basic_classification,  # Basic classification function
        "model_options": {"temperature": 0},  # Lower temperature for deterministic output
        "binary_classification": False, # 4-class classification (not binary)
        "is_fewshot":True
    },
            {
        "model": "llama3.2:3b",  # Using the 3B version of the model
        "prompt_generator": cot_prompt,  # Zero-shot classification
        "prompt_type":"_cot_zeroshot",
        "classification_func": basic_classification,  # Basic classification function
        "model_options": {"temperature": 0},  # Lower temperature for deterministic output
        "binary_classification": False, # 4-class classification (not binary)
        "is_fewshot":False
    },
           {
        "model": "llama3.2:3b",  # Using the 3B version of the model
        "prompt_generator": cot_prompt,  # Zero-shot classification
        "prompt_type":"_cot_fewshot",
        "classification_func": basic_classification,  # Basic classification function
        "model_options": {"temperature": 0},  # Lower temperature for deterministic output
        "binary_classification": False, # 4-class classification (not binary)
        "is_fewshot":True
    },
      {
        "model": "llama3.2:3b",  # Using the 3B version of the model
        "prompt_generator": structured_prompt,  # Zero-shot classification
        "prompt_type":"_str_zeroshot",
        "classification_func": basic_classification,  # Basic classification function
        "model_options": {"temperature": 0},  # Lower temperature for deterministic output
        "binary_classification": False, # 4-class classification (not binary)
        "is_fewshot":False
    },
    
   
]
#     print(experiment_configs)
    
    # Run all experiments
    run_experiments(input_csv, experiment_configs)

In [ ]:
main()

Starting classification of 10 commits...
Processed 10/10 commits (100.0%)
Classification complete. Results saved to commits_classified_llama3.2_3b__basic_zeroshot.csv
Starting classification of 10 commits...
Processed 10/10 commits (100.0%)
Classification complete. Results saved to commits_classified_llama3.2_3b__basic_fewshot.csv
Starting classification of 10 commits...
Processed 10/10 commits (100.0%)
Classification complete. Results saved to commits_classified_llama3.2_3b__cot_zeroshot.csv
Starting classification of 10 commits...


In [ ]:
!ls

In [139]:
pd.read_csv('commits_classified_llama3.2_3b_temp0_few_binary_exp0.csv')

,url,original_message,message,original_label,is_good,is_bad,binary_label,model_classification
0,https://github.com/junit-team/junit4/commit/12...,Mark ThreadGroups created by FailOnTimeout as ...,Mark ThreadGroups created by FailOnTimeout as ...,Why and What,True,False,1.0,both why and what
1,https://github.com/junit-team/junit4/commit/43...,Use Google's Maven Central mirror \n \n We ar...,Use Google's Maven Central mirror\nWe are usin...,Why and What,True,False,1.0,both why and what
2,https://github.com/junit-team/junit4/commit/6c...,Remove reference to obsolete mailing list,Remove reference to obsolete mailing list,No Why,False,False,NaN,no what
3,https://github.com/junit-team/junit4/commit/46...,Deprecate Assert#assertThat \n \n The method ...,Deprecate Assert#assertThat\nThe method 'asser...,Why and What,True,False,1.0,both why and what
4,https://github.com/junit-team/junit4/commit/8a...,Revert 'ExternalResource: declare after() to t...,Revert 'ExternalResource: declare after() to t...,Why and What,False,False,NaN,no why
5,https://github.com/junit-team/junit4/commit/37...,spelling: unwieldy,spelling: unwieldy,Why and What,False,True,0.0,no what
6,https://github.com/junit-team/junit4/commit/b9...,spelling: methods,spelling: methods,Why and What,False,True,0.0,no what
7,https://github.com/junit-team/junit4/commit/a5...,spelling: builder,spelling: builder,Why and What,False,True,0.0,no what
8,https://github.com/junit-team/junit4/commit/21...,spelling: assert,spelling: assert,Why and What,False,True,0.0,no what
9,https://github.com/junit-team/junit4/commit/03...,style: remove space,style: remove space,Why and What,False,True,0.0,no what
